In [10]:
import pandas as pd
import os
pd.set_option('display.float_format', '{:.3f}'.format)

In [11]:
OUTPUT_DIR = "./trained_on_v1/output_csvs"
for filename in os.listdir(OUTPUT_DIR):
    if filename.endswith(".csv"):
        print(filename)

eval_trained_on_dataset_v1.csv


# Metrics taken in account

## Accuracy

Accuracy is calculated as:

$$
\text{Accuracy} = \frac{\text{True Positive + True Negative}}{\text{True Positive + False Positive + True Negative + False Negative}}
$$


## Precision

The fraction of produced detections which are true positives.

$$
\text{Precision} = \frac{\text{True Positive}}{\text{True Positive + False Positive}} = \frac{\text{True Positive}}{\text{Total Number of Prediction}}
$$


## Recall

The fraction of groundtruth boxes in the data that matched to some produced detection.

$$
\text{Recall} = \frac{\text{True Positive}}{\text{True Positive + False Negative}} = \frac{\text{True Positive}}{\text{Total Number of Ground Truth}}
$$

## F1 score

F1 calcula el balance entre precision y recall. Si el F1 es alto, precision y recall son altos.

The F1 Score is calculated as:

$$
\text{F1 Score} = \frac{2 \times (\text{Precision} \times \text{Recall})}{\text{Precision} + \text{Recall}}
$$

# The problem

We are dealing with a weapon detection issue. It is more important to avoid false negatives than false positives. It is better not to miss a weapon detection, at the cost of detecting more weapons with less accuracy. For this reason, a low threshold will also be used.

Given the context of weapon detection and the emphasis on avoiding false negatives (missing weapon detections) at the expense of potentially having more false positives (incorrect weapon detections), **recall** is more important than precision in this scenario.

Recall, also known as sensitivity or true positive rate, measures the proportion of actual positive cases (weapons) that were correctly identified by the model. In the context of your problem, a higher recall means that the model is effectively capturing a larger portion of actual weapons, minimizing the risk of missing any potentially dangerous objects.

While precision (positive predictive value) is also significant, it prioritizes the accuracy of positive predictions among all predicted positives. In your case, a focus on precision might lead to being overly cautious and producing fewer false positives, but it could also result in missing actual weapon detections, which is a more critical concern in a weapon detection scenario.

A higher **recall** rate is crucial for ensuring that potential weapons are not overlooked, even if it means accepting a higher number of false positives.

# Evaluating models vs small dataset
## Training 50 epochs

In [13]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v1.csv")
# df.set_index('model_key', inplace=True)

columns_of_interest = ['model','imgsz','transfer_learning', 'lr0', 'optimizer', 'all_P', 'all_R',
                       'all_F1', 'all_mAP@.5', 'all_mAP@.5:.95']

df[(df['epochs'] == 50)][columns_of_interest].sort_values(by=['all_P'])

,model,imgsz,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
1,yolov5,800,no,0.010,SGD,0.737,0.625,0.338,0.701,0.358
4,yolov5,640,yolov5s,0.010,SGD,0.773,0.712,0.371,0.764,0.429
6,yolov5,800,yolov5s,0.010,SGD,0.796,0.716,0.377,0.755,0.428
9,yolov7,640,yolov7training,0.001,SGD,0.839,0.902,0.435,0.907,0.626
12,yolov8,800,no,0.010,SGD,0.842,0.744,0.395,0.841,0.579
18,yolov8,800,yolov8s,0.010,SGD,0.864,0.776,0.409,0.852,0.583
16,yolov8,640,yolov8s,0.001,Adam,0.868,0.782,0.411,0.857,0.595


### Initial conclusions

The experiments were on yolov5, yolov7 and yolov8, and are evaluated on different image sizes, number of epochs, and batch sizes. The first experiments were trained in 50 epochs. 

Among the models, YOLOv8 generally performs better than YOLOv5 and YOLOv7 in terms of precision, but YOLOv7 performs better in recall and in mAP.

#### Transfer learning
Some experiments use transfer learning by starting with pre-trained weights ('yolov5s.pt', 'yolov7_training.pt', 'yolov8s.pt'). Transfer learning seems to generally improve performance in terms of precision, recall, and F1-score.

#### Image size

Smaller image sizes (640 pixels) lead to better performance compared to larger image sizes (800 pixels).

#### Loss function and learning rate

A smaller learning rate leads to more optimal models but takes longer to train. The ADAM optimizer seems to have better accuracy than the SGD optimizer.

## Training 100 epochs 

In [14]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v1.csv")

columns_of_interest = ['model','imgsz','batch', 'transfer_learning', 'lr0', 'optimizer', 'all_P', 'all_R',
                       'all_F1', 'all_mAP@.5', 'all_mAP@.5:.95']

df[(df['epochs'] == 100)][columns_of_interest].sort_values(by=['all_R'])

,model,imgsz,batch,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
2,yolov5,640,16,yolov5s,0.001,SGD,0.692,0.603,0.322,0.665,0.333
0,yolov5,800,16,no,0.010,SGD,0.793,0.689,0.369,0.747,0.422
5,yolov5,800,16,yolov5s,0.010,SGD,0.854,0.745,0.398,0.807,0.490
3,yolov5,640,16,yolov5s,0.010,SGD,0.811,0.750,0.390,0.807,0.476
17,yolov8,800,16,yolov8s,0.010,SGD,0.877,0.756,0.406,0.844,0.612
13,yolov8,800,16,yolov8l,0.010,SGD,0.867,0.758,0.404,0.838,0.597
11,yolov8,800,16,no,0.010,SGD,0.871,0.776,0.410,0.846,0.599
15,yolov8,640,16,yolov8s,0.010,SGD,0.888,0.777,0.414,0.860,0.623
14,yolov8,640,16,yolov8s,0.001,Adam,0.852,0.796,0.412,0.849,0.616
10,yolov7,800,8,yolov7training,0.010,SGD,0.856,0.808,0.416,0.858,0.584


Increasing the number of training epochs from 50 to 100 seems to have improved the performance of most models across various metrics.

Higher epochs might allow the models to learn more complex patterns, leading to better results.

From the table, we can draw the following conclusions:

1. **Size Doesn't Always Mean Better:** The table suggests that having a bigger model does not necessarily guarantee better performance. For instance, the yolov8l model, which is larger than yolov8s, does not perform better.

2. **Impact of Image Size:** For every model, it is shown that having a 640 imgsz gives a better recall.

3. **Model Version:** Different versions of YOLO show differences in performance. Some versions might perform better in terms of certain metrics. In terms of recall, yolov7 performs better than yolov5 and yolov8. In terms of precision, the best model was achieved with yolov8.

4. **Transfer learning:** Across various configurations, using transfer learning consistently yields higher precision (P), recall (R), F1-score (F1), and mean Average Precision (mAP) metrics compared to non-transfer learning. This highlights the effectiveness of leveraging pre-trained models for better object detection performance.

5. **Precision and Recall Trade-off:** It's clear that there's a trade-off between precision and recall. Some models have higher precision but lower recall, while others have higher recall but lower precision.

Besides the metrics, the practical aspects like computational resources and inference speed will be considered choosing a model for deployment.

# Training models on a bigger dataset

Ideally, considering the previous evaluations, the optimal choice for training models on a larger dataset would have been yolov7. However, due to resource limitations on the training platform (Google Colab), specifically in terms of memory allocation, it proved unfeasible. Attempts to train the models on a bigger dataset resulted in memory overflows, abruptly terminating the training process before the model could be effectively trained... So, the next option was yolov8.

- image size: 640
- batch size: 16
- transfer learning: yolov8s
- lr: between 0.001 (with Adam) and 0.01 (with SGD)
- epochs: 100 and 300, to test if training longer gives better results

In [43]:
OUTPUT_DIR = "./trained_on_v2/output_csvs"
for filename in os.listdir(OUTPUT_DIR):
    if filename.endswith(".csv"):
        print(filename)

eval_trained_on_dataset_v2.csv


In [44]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v2.csv")

columns_of_interest = ['model','imgsz','epochs', 'transfer_learning', 'lr0', 'optimizer', 'all_P', 'all_R',
                       'all_F1', 'all_mAP@.5', 'all_mAP@.5:.95']

df[columns_of_interest].sort_values(by=['epochs','all_mAP@.5:.95'])

,model,imgsz,epochs,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
7,yolov8,800,40,yolov8s,0.010,SGD,0.997,0.924,0.480,0.980,0.932
4,yolov8,640,50,yolov8s,0.001,Adam,0.997,0.955,0.488,0.979,0.932
1,yolov8,640,50,yolov8m,0.010,SGD,0.991,0.962,0.488,0.974,0.938
6,yolov8,800,60,yolov8s,0.010,SGD,0.993,0.962,0.489,0.977,0.946
8,yolov8,800,80,yolov8s,0.010,SGD,0.987,0.961,0.487,0.982,0.948
0,yolov8,640,100,yolov8m,0.010,SGD,0.993,0.962,0.489,0.977,0.941
2,yolov8,640,100,yolov8s,0.001,Adam,0.986,0.962,0.487,0.981,0.941
5,yolov8,800,100,yolov8s,0.010,SGD,0.987,0.961,0.487,0.982,0.949
3,yolov8,640,300,yolov8s,0.010,SGD,1.000,0.961,0.490,0.982,0.942


In [26]:
# df[(df['dataset'] == 'v2') & (df['epochs'] == 50)][columns_of_interest].sort_values(by=['all_R'])

In [45]:
OUTPUT_DIR = "./eval_on_randomized_clips/output_csvs"
for filename in os.listdir(OUTPUT_DIR):
    if filename.endswith(".csv"):
        print(filename)

eval_on_randomized_clips.csv


In [47]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_on_randomized_clips.csv")

df[columns_of_interest].sort_values(by=['all_mAP@.5:.95'])

,model,imgsz,epochs,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
4,yolov8,640,50,yolov8s,0.001,Adam,0.307,0.302,0.152,0.214,0.083
2,yolov8,640,100,yolov8s,0.001,Adam,0.435,0.286,0.173,0.269,0.102
6,yolov8,800,40,yolov8s,0.010,SGD,0.372,0.327,0.174,0.268,0.110
1,yolov8,640,50,yolov8m,0.010,SGD,0.331,0.345,0.169,0.285,0.114
8,yolov8,800,80,yolov8s,0.010,SGD,0.560,0.302,0.196,0.310,0.123
5,yolov8,800,100,yolov8s,0.010,SGD,0.578,0.302,0.198,0.314,0.124
7,yolov8,800,60,yolov8s,0.010,SGD,0.581,0.293,0.195,0.318,0.125
3,yolov8,640,300,yolov8s,0.010,SGD,0.509,0.311,0.193,0.325,0.137
0,yolov8,640,100,yolov8m,0.010,SGD,0.576,0.335,0.212,0.366,0.145
